In [1]:
%load_ext autoreload
%autoreload 2

# mount drive
from google.colab import drive
drive.mount('/content/drive')

# cd into project directory
%cd /content/drive/My\ Drive/Georgia_Tech/Spring_2021/sbic_stereotypes/src

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1M873oJOlIb1Bd5vliq6d04b61XsHLT5h/sbic_stereotypes/src


In [2]:
!pip install transformers
!pip install datasets

import torch
import pandas as pd
import numpy as np

# Useful constants
CLASSIFIER_MODEL_NAME = 'bert-base-uncased'
CLASSIFIERS = ['./classification/model/whoTarget/checkpoint-1280/']
#CLASSIFIERS = ['./classification/model/offensiveYN/checkpoint-898/',
#               './classification/model/intentYN/checkpoint-898/',
#               './classification/model/sexYN/checkpoint-898/',
#               './classification/model/whoTarget/checkpoint-1280/']
SEQ2SEQ_MODEL_NAME = 'facebook/bart-base'
SEQ2SEQ_MODEL = 'model/checkpoint-10782/'
DATA_DIR = '../data/'

     |████████████████████████████████| 2.2MB 7.6MB/s 
     |████████████████████████████████| 3.3MB 43.2MB/s 
     |████████████████████████████████| 870kB 52.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=c7f88a8d0fa287c8ee683f8495bb422dd2e455f7de579ac92402d67fc107ff50
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 194kB 8.9MB/s 
     |████████████████████████████████| 245kB 14.3MB/s 
     |████████████████████████████████| 112kB 13.8MB/s 


In [4]:
from seq2seq_utils import *
from torch import nn, torch
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BartTokenizer, BartForConditionalGeneration

df = pd.read_csv(DATA_DIR + 'SBIC.v2.dev.csv')
clean_post(df)
df = clean_target(df)
df.to_csv('data/clean_dev_df.csv')

dataset = Dataset.from_pandas(df)
seq2seq_tok, classifier_tok, tokenized = get_tokenized_data(
    dataset,
    SEQ2SEQ_MODEL_NAME,
    CLASSIFIER_MODEL_NAME
)

print(tokenized)



Dataset({
    features: ['__index_level_0__', 'attention_mask', 'classifier_attention', 'classifier_inputs', 'input_ids', 'token_type_ids'],
    num_rows: 4680
})


In [6]:
from transformers import BartForConditionalGeneration
#from seq2seq import BartForConditionalGenerationJoinModel
import torch

### MODEL START CODE ###

### MODEL END CODE ###
model = BartForConditionalGenerationJoinModel.from_pretrained(
            SEQ2SEQ_MODEL_NAME,
            classifiers=CLASSIFIERS,
        )
model.eval();

100%|██████████| 1/1 [00:06<00:00,  6.73s/it]
Some weights of BartForConditionalGenerationJoinModel were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['v']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
i = 345
j = 350

#input_ids = torch.tensor([tokenized['input_ids'][i]])
#attention_mask = torch.tensor([tokenized['attention_mask'][i]])
#classifier_inputs = torch.tensor([tokenized['classifier_inputs'][i]])
#classifier_attention = torch.tensor([tokenized['classifier_attention'][i]])

input_ids = torch.tensor(tokenized['input_ids'][i:j])
attention_mask = torch.tensor(tokenized['attention_mask'][i:j])
classifier_inputs = torch.tensor(tokenized['classifier_inputs'][i:j])
classifier_attention = torch.tensor(tokenized['classifier_attention'][i:j])
num_beams = 10

encoder_outputs = model.get_encoder()(
    input_ids,
    attention_mask,
    return_dict=True,
)

kwargs = {
    'attention_mask':attention_mask,
    'classifier_inputs': classifier_inputs,
    'classifier_attention': classifier_attention,
    'beam_search': num_beams > 1,
    'encoder_outputs': encoder_outputs,
}

#for i in range(5):
#  model(input_ids=input_ids, attention_mask=attention_mask, classifier_inputs=classifier_inputs, classifier_attention=classifier_attention);
outputs = model.generate(input_ids, num_beams=num_beams, **kwargs)
seq2seq_tok.decode(outputs[0])

Input Ids:  torch.Size([50, 1])
Input Ids:  torch.Size([50, 2])
Input Ids:  torch.Size([50, 3])
Input Ids:  torch.Size([50, 4])
Input Ids:  torch.Size([50, 5])
Input Ids:  torch.Size([50, 6])
Input Ids:  torch.Size([50, 7])
Input Ids:  torch.Size([50, 8])
Input Ids:  torch.Size([50, 9])
Input Ids:  torch.Size([50, 10])
Input Ids:  torch.Size([50, 11])
Input Ids:  torch.Size([50, 12])
Input Ids:  torch.Size([50, 13])
Input Ids:  torch.Size([50, 14])
Input Ids:  torch.Size([50, 15])
Input Ids:  torch.Size([50, 16])
Input Ids:  torch.Size([50, 17])
Input Ids:  torch.Size([50, 18])
Input Ids:  torch.Size([50, 19])


'</s> Protest Protest Protest Superintendent REUTERS REUTERS REUTERS Protest Protest REUTERS Protest REUTERS REUTERS Superintendent REUTERS ProtestÂ REUTERS</s>'